In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import scipy.misc, numpy, shutil, os, nibabel
import sys, getopt
import zipfile, os

In [ ]:
#import congitive assessment spreadsheet
#path = "./Data/matthewl225_7_15_2019_16_11_22.csv"
path = input('Path to cognitive assesment csv: ')
header = ['ADRC_ADRCCLINICALDATA ID', 'dx1']

#dfcolumns = pd.read_csv(path, nrows = 1)
df = pd.read_csv(path, header = None, skiprows = 1, usecols = [0,8], names = header)

#df = pd.read_csv(path, names=header, index_col=False)

In [ ]:
df.shape

In [ ]:
df[:3]

In [ ]:
df.loc[[4065]]

In [ ]:
df.dx1.unique()

In [ ]:
df.dx1.isna().sum()

In [ ]:


subject_regex = re.compile("OAS(?P<order>[0-9]+)")
subjects = [subject_regex.search(r).group(1) for r in df["ADRC_ADRCCLINICALDATA ID"]]
#print(subjects)
df['Subject'] = pd.Series(subjects, index=df.index)

date_regex = re.compile("d(?P<order>[0-9]+)")
dates = [date_regex.search(r).group(1) for r in df["ADRC_ADRCCLINICALDATA ID"]]
df['Date'] = pd.Series(list(map(int,dates)), index=df.index)

In [ ]:
df[:15]

In [ ]:
df.tail(10)

In [ ]:
#unzip nii package
package_path = input('Path to nii zip: ')
with zipfile.ZipFile(package_path, "r") as zip_ref:
    zip_ref.extractall("./Data/Nii")

In [ ]:
import gzip

#unzip individual run nii pakcages
nii_package_path = "./Data/Nii"
output_path = "./Data/Nii2"

if not os.path.exists(output_path):
    os.makedirs(output_path)
    print("Created ouput directory: " + output_path)

for root, directories, filenames in os.walk(nii_package_path):
    for filename in filenames:
        run_package = os.path.join(root, filename)
        with gzip.open(run_package, 'rb') as s_file, open(os.path.join(output_path, filename[:-3]), 'wb') as d_file:
            shutil.copyfileobj(s_file, d_file, 65536)
            #shutil.move(d_file, output_path)
                

In [ ]:
shutil.rmtree("./Data/Nii")

In [ ]:
#convert nii to png (borrowed from nii2png.py)
outputfile = "./Data/Images"
rotate_angle = int(input('Enter rotate angle (0 90 180 270): '))

if not (rotate_angle == 0 or rotate_angle == 90 or rotate_angle == 180 or rotate_angle == 270):
    print('You must enter a value that is either 90, 180, or 270. Quitting...')
    sys.exit()

data_path = "./Data/Nii2"
for root, directories, filenames in os.walk(data_path):
    for filename in filenames:
        if "T2" in filename or "echo" in filename:
            print(filename)
            continue
        
        nii_path = os.path.join(root, filename)
        image_array = nibabel.load(nii_path).get_data()
        print("Input file is ", nii_path)
        #print(len(image_array.shape))

        if len(image_array.shape) == 3:
            # set 4d array dimension values
            nx, ny, nz = image_array.shape

            if not os.path.exists(outputfile):
                os.makedirs(outputfile)
                print("Created ouput directory: " + outputfile)
            
            image_folder = os.path.join(outputfile, filename[:-8])
            
            if not os.path.exists(image_folder):
                os.makedirs(image_folder)
                print("Created ouput directory: " + image_folder)
            
            print('Reading NIfTI file...')

            total_slices = image_array.shape[2]

            slice_counter = 0
            # iterate through slices
            for current_slice in range(0, total_slices):
                # alternate slices
                if (slice_counter % 1) == 0:
                    # rotate or no rotate
                    if rotate_angle == 90 or rotate_angle == 180 or rotate_angle == 270:
                        if rotate_angle == 90:
                            data = numpy.rot90(image_array[:, :, current_slice])
                        elif ask_rotate_num == 180:
                            data = numpy.rot90(numpy.rot90(image_array[:, :, current_slice]))
                        elif ask_rotate_num == 270:
                            data = numpy.rot90(numpy.rot90(numpy.rot90(image_array[:, :, current_slice])))
                    else:
                        data = image_array[:, :, current_slice]

                    #alternate slices and save as png
                    if (slice_counter % 1) == 0:
                        #print('Saving image...')
                        image_name = nii_path[:-4] + "_z" + "{:0>3}".format(str(current_slice+1))+ ".png"
                        scipy.misc.imsave(image_name, data)
                        #print('Saved.')

                        #move images to folder
                        #print('Moving image...')
                        src = image_name
                        shutil.move(src, image_folder)
                        slice_counter += 1
                        #print('Moved.')

            print('Finished converting {}'.format(filename))
        else:
            print('Not a 3D Image. Please try again.')
        




In [ ]:
runs_list = next(os.walk('./Data/Images'))[1]
#print(runs_list)
print(len(runs_list))

In [ ]:
import math

AD_list = []

for run in list(runs_list):
    # if dx1 starts with AD then patient has AD
    # either 0 or 1
    # print(run)
    subject_regex = re.compile("OAS(?P<order>[0-9]+)")
    subject = subject_regex.search(run).group(1)
    
    date_regex = re.compile("d(?P<order>[0-9]+)")
    date = date_regex.search(run).group(1) 
    
    #print(subject, date)
    
    subject_df = df.loc[df["Subject"] == subject]
    #print(subject_df)
    
    #label = min(list(map(int, subject_df['Date'])), key=lambda x:(int(date)-x))
    df_dates = subject_df['Date']
    #print(type(df_dates))
    
    if int(date) > df_dates.iloc[-1]:
        label_date = df_dates.iloc[-1]
    else:
        label_date = min(i for i in df_dates if i >= int(date))
    
    label = subject_df.loc[df['Date'] == label_date, 'dx1'].item()
    if type(label) == float:
        print(subject, label)
        runs_list.remove(run)
        continue
        
#     if type(label) != str:
#         print(type(label))

    AD = 1 if label.startswith('AD') else 0
    AD_list.append(AD)
    
    #print(run, AD)

print('runs_list len:')
print(len(runs_list))
print('AD_list len:')
print(len(AD_list))


# # Validation
# for idx, val in enumerate(AD_list):
#     subject_regex = re.compile("OAS(?P<order>[0-9]+)")
#     subject = subject_regex.search(runs_list[idx]).group(1)
#     subject_df = df.loc[df["Subject"] == subject]
#     isAD = 0
#     for index, row in subject_df.iterrows():
#         if  row['dx1'].startswith('AD'):
#             isAD = 1
#             break
#     if isAD != val:
#         print(subject, val, isAD)
#         print('Invalid match found')
#         break
#     print(subject, val, isAD)
    


Labelsdf = pd.DataFrame()
#df = pd.DataFrame(columns=['Run', 'AD'])
Labelsdf['Run'] = pd.Series(runs_list)
Labelsdf['AD'] = pd.Series(AD_list)

In [ ]:
print(label)

In [ ]:
Labelsdf

In [ ]:
Labelsdf.to_csv("./Data/Labels.csv")

In [ ]:
Labelsdf.groupby("AD").size()